In [ ]:
import numpy as np

# Saaty's scale for pairwise comparisons
SAATY_SCALE = {
    1: "Equal importance",
    2: "Weak or slight importance",
    3: "Moderate importance",
    4: "Moderate plus importance",
    5: "Strong importance",
    6: "Strong plus importance",
    7: "Very strong or demonstrated importance",
    8: "Very, very strong importance",
    9: "Extreme importance"
}

# Random Index (RI) values for consistency check (for n=1 to 10)
RI_VALUES = [0.00, 0.00, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49]

def print_saaty_scale():
    print("\nSaaty's Scale for Pairwise Comparisons:")
    for value, desc in SAATY_SCALE.items():
        print(f"{value}: {desc}")
    print("Use reciprocals (1/value) for inverse comparisons.\n")

def get_pairwise_matrix(n, names, is_criteria=True):
    """
    Get pairwise comparison matrix from user input.
    """
    matrix = np.ones((n, n))
    print_saaty_scale()
    item_type = "criteria" if is_criteria else "alternatives"
    print(f"Enter pairwise comparisons for {item_type}:")
    for i in range(n):
        for j in range(i + 1, n):
            while True:
                try:
                    value = float(input(f"How much more important is '{names[i]}' over '{names[j]}'? (1-9 or 1/1-1/9): "))
                    if value <= 0:
                        print("Value must be positive. Try again.")
                        continue
                    matrix[i, j] = value
                    matrix[j, i] = 1 / value
                    break
                except ValueError:
                    print("Invalid input. Please enter a number.")
    return matrix

def calculate_weights(matrix):
    """
    Calculate priority weights using eigenvector method.
    """
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eig_index = np.argmax(eigenvalues)
    priorities = eigenvectors[:, max_eig_index].real
    priorities /= np.sum(priorities)  # Normalize
    return priorities

def consistency_check(matrix):
    """
    Calculate Consistency Index (CI) and Consistency Ratio (CR).
    """
    n = matrix.shape[0]
    eigenvalues = np.linalg.eigvals(matrix)
    lambda_max = np.max(eigenvalues).real
    CI = (lambda_max - n) / (n - 1)
    RI = RI_VALUES[n - 1] if n <= 10 else 1.49  # Approximate for n>10
    CR = CI / RI if RI != 0 else 0
    return CI, CR

def main():
    print("Welcome to the Interactive AHP Calculator!")
    
    # Step 1: Criteria
    num_criteria = int(input("Enter the number of criteria: "))
    criteria_names = []
    for i in range(num_criteria):
        name = input(f"Enter name for criterion {i+1}: ")
        criteria_names.append(name)
    
    criteria_matrix = get_pairwise_matrix(num_criteria, criteria_names, is_criteria=True)
    criteria_weights = calculate_weights(criteria_matrix)
    ci, cr = consistency_check(criteria_matrix)
    print("\nCriteria Pairwise Matrix:")
    print(criteria_matrix)
    print("\nCriteria Weights:")
    for name, weight in zip(criteria_names, criteria_weights):
        print(f"{name}: {weight:.4f}")
    print(f"Consistency Ratio (CR): {cr:.4f}")
    if cr > 0.1:
        print("Warning: CR > 0.1, consider revising comparisons for consistency.")
    
    # Step 2: Alternatives
    num_alternatives = int(input("\nEnter the number of alternatives: "))
    alternative_names = []
    for i in range(num_alternatives):
        name = input(f"Enter name for alternative {i+1}: ")
        alternative_names.append(name)
    
    # Step 3: Pairwise for alternatives under each criterion
    alt_matrices = []
    alt_weights = []
    for crit_idx, crit_name in enumerate(criteria_names):
        print(f"\nPairwise comparisons for alternatives under criterion '{crit_name}':")
        matrix = get_pairwise_matrix(num_alternatives, alternative_names, is_criteria=False)
        weights = calculate_weights(matrix)
        ci, cr = consistency_check(matrix)
        alt_matrices.append(matrix)
        alt_weights.append(weights)
        print(f"\nAlternatives Matrix for '{crit_name}':")
        print(matrix)
        print(f"Local Weights for '{crit_name}':")
        for name, weight in zip(alternative_names, weights):
            print(f"{name}: {weight:.4f}")
        print(f"Consistency Ratio (CR): {cr:.4f}")
        if cr > 0.1:
            print("Warning: CR > 0.1, consider revising comparisons for consistency.")
    
    # Step 4: Synthesize global priorities
    global_priorities = np.zeros(num_alternatives)
    for i in range(num_alternatives):
        for j in range(num_criteria):
            global_priorities[i] += criteria_weights[j] * alt_weights[j][i]
    
    # Normalize global priorities (though they should already sum to 1)
    global_priorities /= np.sum(global_priorities)
    
    # Step 5: Output results
    print("\nFinal Global Priorities and Ranking:")
    ranked = sorted(zip(alternative_names, global_priorities), key=lambda x: x[1], reverse=True)
    for rank, (name, priority) in enumerate(ranked, 1):
        print(f"Rank {rank}: {name} with priority {priority:.4f}")

if __name__ == "__main__":
    main()

Welcome to the Interactive AHP Calculator!
